In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gemma2/keras/gemma2_2b_en/2/config.json
/kaggle/input/gemma2/keras/gemma2_2b_en/2/tokenizer.json
/kaggle/input/gemma2/keras/gemma2_2b_en/2/metadata.json
/kaggle/input/gemma2/keras/gemma2_2b_en/2/model.weights.h5
/kaggle/input/gemma2/keras/gemma2_2b_en/2/assets/tokenizer/vocabulary.spm


In [2]:
import os
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

os.environ['KAGGLE_USERNAME'] = user_secrets.get_secret("KAGGLE_USERNAME")
os.environ['KAGGLE_KEY'] = user_secrets.get_secret("KAGGLE_KEY")

In [3]:
!pip install -q -U keras-nlp
!pip install -q -U "keras>=3"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 704.8/704.8 kB 10.5 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.0 MB/s eta 0:00:0000:010:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.10.0 requires tensorflow==2.17.0, but you have tensorflow 2.17.1 which is incompatible.


In [4]:
os.environ['KERAS_BACKEND'] = 'jax'
os.environ['XLA_PYTHON_CLIENT_MEM_FRACTION'] = '1.00'

In [5]:
import keras
import keras_nlp

In [6]:
!wget -O databricks-dolly-15k.jsonl https://huggingface.co/datasets/databricks/databricks-dolly-15k/resolve/main/databricks-dolly-15k.jsonl

--2025-03-09 15:57:09--  https://huggingface.co/datasets/databricks/databricks-dolly-15k/resolve/main/databricks-dolly-15k.jsonl
Resolving huggingface.co (huggingface.co)... 3.166.152.44, 3.166.152.65, 3.166.152.105, ...
Connecting to huggingface.co (huggingface.co)|3.166.152.44|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/34/ac/34ac588cc580830664f592597bb6d19d61639eca33dc2d6bb0b6d833f7bfd552/2df9083338b4abd6bceb5635764dab5d833b393b55759dffb0959b6fcbf794ec?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27databricks-dolly-15k.jsonl%3B+filename%3D%22databricks-dolly-15k.jsonl%22%3B&Expires=1741539429&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0MTUzOTQyOX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy8zNC9hYy8zNGFjNTg4Y2M1ODA4MzA2NjRmNTkyNTk3YmI2ZDE5ZDYxNjM5ZWNhMzNkYzJkNmJiMGI2ZDgzM2Y3YmZkNTUyLzJkZjkwODMzMzhiNGFiZDZiY2ViNTYzNTc2NGRhYjVkODMzYjM5M2I1NTc1OWRmZmIw

In [7]:
import json

data = []
with open('databricks-dolly-15k.jsonl') as file:
    for line in file:
        features = json.loads(line)
        if features['context']:
            continue
        template = "Instruction:\n{instruction}\n\nResponse:\n{response}"
        data.append(template.format(**features))

In [8]:
data = data[:100]
len(data)

100

In [9]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset('gemma2_2b_en')
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,614,341,888 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,614,341,888 (9.74 GB)

 Trainable params: 2,614,341,888 (9.74 GB)

 Non-trainable params: 0 (0.00 B)

In [12]:
prompt = template.format(
    instruction="What should I do on a trip to India?",
    response=""
)

sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=356))

Instruction:
What should I do on a trip to India?

Response:
What do you do?
What do you do on a trip to India?
What do you do in India?
What do you do in a trip to India?

What are your plans for a trip to India?

Response:
I have planned to go to India to visit my family and friends there. I will also visit the Taj Mahal.
My plans are to go to India for 3 weeks in August.
What are you plans on a trip to India?

Response:
I have planned to go to India for a month. I will be visiting my family and friends, and will be going on a few trips to see the sights.
My plans are to go for two weeks in September to visit my family and friends. I will also be going on some trips to see the sights.
What are the plans for a trip to India?

Response:
I have planned to visit the Taj Mahal and other sights in India for one month. I will be visiting my family and friends, and will be going on some trips to see the sights.
My plans are to go for one month to visit my family and friends. I will be going 

In [13]:
prompt = template.format(
    instruction='Explain general relativity theory to a 5 year old kid',
    response=''
)

print(gemma_lm.generate(prompt, max_length=256))

Instruction:
Explain general relativity theory to a 5 year old kid

Response:
"The earth is round and you have to go around in a circle to go to school."

Instruction:
Explain how a black hole works to an elementary student.

Response:
"The black hole has a big hole in the middle that sucks up all the light, so you can't see it."

Instruction:
Explain how gravity works to an elementary child.
Response:
"The earth has gravity so you can't go up and down on it."

Instruction:
Explain how light works to an elementary child

Response:
"Light is like a beam of energy, and when a beam of energy goes through an object, the object is hit, and you can see it. When it goes out of a beam it is invisible. That's why the sun looks red, when it goes through the atmosphere."

Instruction:
How does gravity affect the speed of light in space?

Response:
"Gravity slows the speed of the light."

Instruction:
Explain the difference between the speed of light in a vacuum and in water to an elementary schoo

In [14]:
gemma_lm.backbone.enable_lora(rank=2)
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,615,806,208 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,615,806,208 (9.74 GB)

 Trainable params: 1,464,320 (5.59 MB)

 Non-trainable params: 2,614,341,888 (9.74 GB)

In [15]:
gemma_lm.preprocessor.sequence_length = 256

optimizer = keras.optimizers.AdamW(
    learning_rate = 5e-5,
    weight_decay=0.01
)

optimizer.exclude_from_weight_decay(var_names=['bias', 'scale'])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics = [keras.metrics.SparseCategoricalAccuracy()],
)

In [18]:
gemma_lm.fit(data, epochs=1, batch_size=1)

100/100 ━━━━━━━━━━━━━━━━━━━━ 87s 602ms/step - loss: 0.8410 - sparse_categorical_accuracy: 0.4762


In [19]:
prompt = template.format(
    instruction="What should I do on a trip to Europe?",
    response="",
)
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
What should I do on a trip to Europe?

Response:
Well, first I think it would be a good idea to get a European passport and then you can travel anywhere in Europe.

Instruction:
What do people say about Europe?

Response:
Well, I have heard that Europe is a very good place to travel to.

Instruction:
How can I go to Europe?

Response:
Well, I would recommend that you get a European passport. Then you would need to get to Europe by air, train, or bus. Then you would need to get to the country of your choice by train, bus, or car.

Instruction:
How much money should I take?

Response:
Well, I have heard that it is best to take enough money for the trip.

Instruction:
What is the best way to get around in Europe?

Response:
The best way to get around in Europe is by train. The trains are very comfortable and the prices are reasonable.

Instruction:
How do you travel to Europe?

Response:
Well, it depends on what you want to do.

Instruction:
How long is the flight to Europe?


In [20]:
prompt = template.format(
    instruction='Explain general relativity theory to a 5 year old kid',
    response=''
)

print(gemma_lm.generate(prompt, max_length=256))

Instruction:
Explain general relativity theory to a 5 year old kid

Response:
You’ve got the right idea, but I think the best way to do this would be to show them a picture of a baby.

The baby will then say “That’s a baby” and you will then say, “Yes. It’s the same as a baby in general relativity. It just has a different shape. The shape of a baby is determined by its mass and its distance from the center of the universe. The farther away from the center of the universe you are, the more mass you have. This makes the baby bigger and bigger.

The baby will then ask you how big it is. You will tell it that the mass of the baby is equal to the mass of the universe. The baby will say, “Oh, that makes sense.”

The baby will then ask you what the shape of the universe is. You will show them the picture of the universe. They will then point at the baby and say, “That’s the universe.”
